# Language Modeling

In this assignment, let's generate text using a trigram language model.

Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>


Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


When you run the `ls` command below, you should see the files in the tweets folder.




In [2]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
covid-tweets-2020-08-10-2020-08-21.trigrams.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


Let's load the trigrams first. You can change the below code as you see fit.

In [3]:
from math import log

bigram_prefix_to_trigram = {}
bigram_prefix_to_trigram_weights = {}

lines = open("/content/drive/My Drive/tweets/covid-tweets-2020-08-10-2020-08-21.trigrams.txt").readlines()
for line in lines:
  word1, word2, word3, count = line.strip().split()

  #initiating new entries if the bigram was not used yet
  if (word1, word2) not in bigram_prefix_to_trigram:
    bigram_prefix_to_trigram[(word1, word2)] = []
    bigram_prefix_to_trigram_weights[(word1, word2)] = []

  #two dictionaries, have bigram as key and array of 3rd words or counts as value
  bigram_prefix_to_trigram[(word1, word2)].append(word3)
  bigram_prefix_to_trigram_weights[(word1, word2)].append(int(count))


# freeup memory
lines = None


## Problem 1: Retrieve top next words and their probability given a bigram prefix.

For the following prefixes **word1=middle, word2=of, and n=10**, the output is:



```
a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
...
...
...
```



In [19]:
def top_next_word(word1, word2, n):

  #add up the counts of all third words that follow the bigram
  total_count=0
  for i in range(len(bigram_prefix_to_trigram_weights[(word1, word2)])):
    total_count += bigram_prefix_to_trigram_weights[(word1, word2)][i]

  #create empty lists that will be returned
  next_words=[]
  probs=[]

  #iterate through n top elements of the bigram's arrays
  for i in range(n):
    #store the third word
    next_words.append(bigram_prefix_to_trigram[(word1, word2)][i])
    #store the probability as the word's count / total count
    probs.append(bigram_prefix_to_trigram_weights[(word1, word2)][i]/total_count)  

  return next_words, probs

next_words, probs = top_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 0.807981220657277
the 0.06948356807511737
pandemic 0.023943661971830985
this 0.016901408450704224
an 0.0107981220657277
covid 0.009389671361502348
nowhere 0.008450704225352112
it 0.004694835680751174
lockdown 0.002347417840375587
summer 0.002347417840375587


## Problem 2: Sampling n words

Sample next n words given a bigram prefix. Use the probablity distribution defined by the frequency counts. Functions like **numpy.random.choice** will be useful here. Sample without repitition, otherwise all your samples will contain the most frequent trigram.


For the following prefixes **word1=middle, word2=of, and n=10**, the output could be as follows (our outputs may differ): 

```
a 0.807981220657277
pandemic 0.023943661971830985
nowhere 0.008450704225352112
the 0.06948356807511737
...
...
...
...
...
```



In [31]:
import numpy

def sample_next_word(word1, word2, n):

  #add up the counts of all third words that follow the bigram
  total_count=0
  for i in range(len(bigram_prefix_to_trigram_weights[(word1, word2)])):
    total_count += bigram_prefix_to_trigram_weights[(word1, word2)][i]

  #create a probability distribution list
  prob_distribution=[]
  for i in range(len(bigram_prefix_to_trigram_weights[(word1, word2)])):
    prob_distribution.append(bigram_prefix_to_trigram_weights[(word1, word2)][i]/total_count)  

  #generate list of indexes for the sample
  #choosing n indexes within the size of the array, without replacement, using probability distribution for picking
  if (n <= len(bigram_prefix_to_trigram_weights[(word1, word2)])):
    list_of_indexes = numpy.random.choice(len(bigram_prefix_to_trigram_weights[(word1, word2)]), n, replace=False, p=prob_distribution)
  else:
    list_of_indexes = numpy.random.choice(len(bigram_prefix_to_trigram_weights[(word1, word2)]), len(bigram_prefix_to_trigram_weights[(word1, word2)]), replace=False, p=prob_distribution)

  #create empty lists that will be returned
  next_words=[]
  probs=[]

  #iterate through the entries based on the created list of indexes
  for i in list_of_indexes:
    #store the third word
    next_words.append(bigram_prefix_to_trigram[(word1, word2)][i])
    #store the probability as the word's count / total count
    probs.append(bigram_prefix_to_trigram_weights[(word1, word2)][i]/total_count)  

  return next_words, probs
  # write your code here

next_words, probs = sample_next_word("middle", "of", 10)
for word, prob in zip(next_words, probs):
  print(word, prob)

a 0.807981220657277
pandemic 0.023943661971830985
the 0.06948356807511737
endorsement 0.00046948356807511736
this 0.016901408450704224
an 0.0107981220657277
#covid19 0.0018779342723004694
no 0.0009389671361502347
that 0.00046948356807511736
summer 0.002347417840375587


## Problem 3: Generate sentences starting with a prefix

Generates n-sentences starting with a given sentence prefix. Use [beam search](https://en.wikipedia.org/wiki/Beam_search) to generate multiple sentences. Depending on which method you use to generate next word, you will get different outputs. When you generate <EOS> in a path, stop exploring that path.

If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output is as follows:
```
<BOS1> <BOS2> trump eyes new unproven coronavirus treatment URL <EOS> 0.00021893147502903603
<BOS1> <BOS2> trump eyes new unproven coronavirus cure URL <EOS> 0.0001719607222046247
<BOS1> <BOS2> trump eyes new unproven virus cure promoted by mypillow ceo over unproven therapeutic URL <EOS> 9.773272077557522e-05
...
...
...
```


If you use the method `word_generator=top_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output is as follows:
```
<BOS1> <BOS2> biden calls for a 30 bonus URL #cashgem #cashappfriday #stayathome <EOS> 0.0002495268686322749
<BOS1> <BOS2> biden says all u.s. governors should mandate masks <EOS> 1.6894510541025754e-05
<BOS1> <BOS2> biden says all u.s. governors question cost of a pandemic <EOS> 8.777606198953028e-07
...
...
...
```


If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> trump`, your output may look as follows (since this is sampling, our outputs will difer):

```
<BOS1> <BOS2> trump signs executive orders URL <EOS> 7.150992253427233e-05
<BOS1> <BOS2> trump signs executive actions URL <EOS> 7.117242889600614e-05
<BOS1> <BOS2> trump news president attacked over it <EOS> 1.0546494007903964e-05
<BOS1> <BOS2> trump news president attacked over executive orders URL <EOS> 1.0126405114118984e-05
```

If you use the method `word_generator=sample_next_word`, `beam=10` and prefix is `<BOS1> <BOS2> biden`, your output may look as follows:

```
<BOS1> <BOS2> biden harris 2020 <EOS> 0.0015758924114719264
<BOS1> <BOS2> biden harris 2020 URL <EOS> 0.0006443960952032196
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do it URL <EOS> 4.105215709355001e-07
<BOS1> <BOS2> biden calls for evictions ban so marylander 's do our best to stay home <EOS> 1.3158806336098573e-09
...
...
...
...
...
```

Hope you see that sampling gives different outputs compared to deterministically picking the top n-words.


In [100]:
sentence ="Hello <EOD>"
sentence_words=sentence.split()
word1=sentence_words[len(sentence_words)-2]
word2=sentence_words[len(sentence_words)-1]
print(word1, word2)

Hello <EOD>


In [157]:
def generate_sentences(prefix, beam, sampler):
  word0, word1, word2 = prefix.strip().split()

  top_sentences=[prefix]
  top_sentence_probs=[1]


  remains = True

  while(remains==True):
    remains=False

    for sentence_i in range(len(top_sentences)):  

      top_sentences[sentence_i] = top_sentences[sentence_i].split()
      word1=top_sentences[sentence_i][len(top_sentences[sentence_i])-2]
      word2=top_sentences[sentence_i][len(top_sentences[sentence_i])-1]
      print(top_sentences[sentence_i])
      print(type(word1), type(word2), word1, word2)
    

      if word2=="<EOS>":
        break
      candidate_sentences=top_sentences[sentence_i]
      candidate_sentence_probs=[]

      #generate candidate words
      if sampler==top_next_word:
        next_words, probs = top_next_word(str(word1), str(word2), beam)
      if sampler==sample_next_word:
        next_words, probs = sample_next_word(word1, word2, beam)

      #create candidate sentences and calculate their probabilities
      for i in range(len(next_words)):
        candidate_sentences.append(next_words[i])
        candidate_sentence_probs.append(top_sentence_probs[sentence_i]*probs[i])
        if next_words[i]!="<EOS>":
          remains=True
        
      #from all candidate sentences created from top sentences, would choose the new top sentences...
      top_sentences = candidate_sentences
      top_sentence_probs = candidate_sentence_probs



  print("Candidate sentences: ", candidate_sentences)
  print("Candidate sentence probs: ", candidate_sentence_probs)


  sentences=[ ]
  probs=[ ]
  return sentences, probs 
  # write your code


sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=2, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=2, sampler=top_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> trump", beam=2, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)
print("#########################\n")

sentences, probs = generate_sentences(prefix="<BOS1> <BOS2> biden", beam=2, sampler=sample_next_word)
for sent, prob in zip(sentences, probs):
  print(sent, prob)

['<BOS1>', '<BOS2>', 'trump']
<class 'str'> <class 'str'> <BOS2> trump
['<BOS1>']
<class 'str'> <class 'str'> <BOS1> <BOS1>


KeyError: ignored

# Semantic Parsing

We will use Wikidata and SPARQL to do semantic parsing, i.e., convert a question to a SPARQL query and execute it on Wikidata to get the answer. First install sparqlwrapper which allows us to write and execute sparql queries.

In [148]:
!pip install sparqlwrapper

     |████████████████████████████████| 235kB 8.5MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 


Here is an example SPARQL query for "countries and their population sorted in descending order". You can play with several example queries here https://query.wikidata.org/ (many examples are also given in this page)


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT DISTINCT ?countryLabel ?population
{
  ?country wdt:P31 wd:Q6256 ;
           wdt:P1082 ?population .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
GROUP BY ?population ?countryLabel
ORDER BY DESC(?population)
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"][:10]:
    print('%s\t%s' % (result["countryLabel"]["value"], result["population"]["value"]))

People's Republic of China	1409517397
India	1326093247
United States of America	328239523
Indonesia	263991379
Pakistan	216565318
Brazil	210147125
Nigeria	190886311
Bangladesh	164669751
Mongol Empire	160000000
Mexico	130526945


The main challenge here is to identify what properties and entities you are interested in. You can use [Wikidata search box](https://www.wikidata.org/wiki/Wikidata:Main_Page) to find an entity id and then finding relevant property ids.


## Problem 4: What are the cities in Quebec?

In [ ]:
# Write your code here


## Problem 5: What is the most populated city in Quebec and its population?

In [ ]:
# Write your code here


## Problem 6: Who are the current premiers of different Canadian provinces sorted by their population?

In [ ]:
# Write your code here


## Problem 7: What is the province in Canada that borders the most number of provinces?

In [ ]:
# Write your code here
